In [1]:
import scipy as sp
import sklearn as sk
import numpy as np
np.random.seed(12345)

In [2]:
q = np.random.randn(3) 
K = np.random.randn(3, 7)
alpha = 1

### Ridge Regression and its gradient descent version

In [3]:
import sklearn.linear_model
ridge = sklearn.linear_model.Ridge(alpha = alpha, fit_intercept=False, random_state=123).fit(K, q)
true_coef = ridge.coef_
v1 = np.linalg.inv(K.T @ K + alpha * np.eye(7)) @ (K.T @ q)
np.testing.assert_allclose(true_coef, v1)
print(v1)
a = np.zeros(7)
L = 20
alpha_max = 3 * alpha
for l in range(1, L + 1):
    kappa = (alpha_max * (alpha/alpha_max) ** (l/L))/(K.shape[0] * K.shape[1])
    a = a - kappa * (K.T @ (K @ a - q) + alpha * a)
    if l % 10 == 0:
        print(l, np.linalg.norm(a - v1) / np.linalg.norm(a))
        # print(a - v1)

[ 0.07510553 -0.11696049  0.09250367  0.04916155  0.08432023 -0.08204641
 -0.04978814]
10 0.21308928631528687
20 0.00028595379934584584


### Subspace Ridge Regression and its Projected Gradient Descent Version

In [4]:
import cvxopt as opt
from cvxopt import matrix
from cvxopt.solvers import qp, options

Test in the simpler case of pure regression without constraints

In [5]:
cv_q = matrix(-K.T @ q)
cv_KK = matrix((K.T @ K + alpha * np.eye(7)))
sol = qp(cv_KK, cv_q)['x']
sol = np.array(sol).squeeze()
np.testing.assert_allclose(sol, v1)

Now add the constraints, we can see that the solution is different from the unconstrained version.

In [6]:
A = matrix(np.ones(7)).T
b = matrix(1.0)
sol = qp(cv_KK, cv_q, A=A, b=b)['x']
sol = np.array(sol).squeeze()

print(sol)
print(sol.sum())

[ 0.38176057 -0.08386346  0.04471772  0.29143839  0.18621987  0.01596309
  0.16376384]
1.0000000000000002


Now execute projected gradient descent.

In [7]:
a = np.zeros(7)
L = 20
alpha_max = 3 * alpha

def proj_subspace(x):
    return x - (x.sum() - 1) / 7

for l in range(1, L + 1):
    kappa = (alpha_max * (alpha/alpha_max) ** (l/L))/(K.shape[0] * K.shape[1])
    a = proj_subspace(a - kappa * (K.T @ (K @ a - q) + alpha * a))
    if l % 10 == 0:
        print(l, np.linalg.norm(a - sol) / np.linalg.norm(sol))

10 0.037875758408875665
20 0.0091313500853822


### Simplex Ridge Regression and its Projected Gradient Version

In [8]:
G = matrix(- np.eye(7))
h = matrix(np.zeros(7))
sol = qp(cv_KK, cv_q, G=G, h=h, A=A, b=b)['x']
sol = np.array(sol).squeeze()
print(sol)
print(sol.sum())

     pcost       dcost       gap    pres   dres
 0: -9.4000e-02 -1.1747e+00  1e+01  3e+00  1e+00
 1: -7.0947e-02 -8.2415e-01  1e+00  8e-02  4e-02
 2: -7.3828e-02 -1.5993e-01  9e-02  6e-03  3e-03
 3: -8.5123e-02 -9.1853e-02  7e-03  1e-04  7e-05
 4: -8.7191e-02 -8.7670e-02  5e-04  3e-06  2e-06
 5: -8.7293e-02 -8.7312e-02  2e-05  3e-08  1e-08
 6: -8.7294e-02 -8.7294e-02  2e-07  3e-10  1e-10
 7: -8.7294e-02 -8.7294e-02  2e-09  3e-12  1e-12
Optimal solution found.
[4.24078454e-01 6.65517269e-11 2.35483901e-02 3.05264533e-01
 1.36881333e-01 1.38425925e-10 1.10227291e-01]
1.0


Based on https://gist.github.com/mblondel/6f3b7aaad90606b98f71 which provides two other methods for projecting onto the simplex

In [11]:
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w


a = np.zeros(7)
L = 20
alpha_max = 3 * alpha

for l in range(1, L + 1):
    kappa = (alpha_max * (alpha/alpha_max) ** (l/L))/(K.shape[0] * K.shape[1])
    a = projection_simplex_sort(a - kappa * (K.T @ (K @ a - q) + alpha * a))
    if l % 10 == 0:
        print(l, np.linalg.norm(a - sol) / np.linalg.norm(sol))

10 0.028416697153768034
20 0.005185129304881217
